# Preprocessing of the dataset  

## Steps :
* Merging the differents datasets into one
* Creating a date column
* Removing column with 70% of null values
* Creating numerical value for the full time result

### Merging the datasets
Merge datasets from season 14-15 to 19-20. Season 20-21 will be kept as testing data so we leave it alone for the moment. 

In [1]:
import os 
import pandas as pd


path = "../assets/data/"
def merged_df():
    
    initial_file = "PL_14-15.csv"
    initial_df = pd.read_csv(path+initial_file)
    files_to_append = [f for f in os.listdir(path) if f != initial_file]
    for f in files_to_append:
        df = pd.read_csv(path+f)
        initial_df = initial_df.append(df)
    return initial_df
df = merged_df()

In [2]:
import numpy as np

## Creating a date column

In [3]:
df["Date"] = pd.to_datetime(df["Date"])

In [4]:
df.isnull().mean() *100

Div               0.021925
Date              0.021925
HomeTeam          0.021925
AwayTeam          0.021925
FTHG              0.021925
                   ...    
MaxCAHA          91.668494
AvgCAHH          91.668494
AvgCAHA          91.668494
Numerical_htr    50.010963
Numerical_ftr    50.010963
Length: 132, dtype: float64

## Removing rows with missing values in critical columns
We cant afford to have missing values inside of FTR, HomeTeam, AwayTeam columns. We remove the rows with missing values at those points

In [5]:
df.dropna(subset=["FTR", "HomeTeam", "AwayTeam"], inplace=True, how="any")

## Removing column with 70% of null values

In [6]:
columns_to_drop = [col for col in df.columns if df[col].isnull().mean() *100 > 75 ]
df.drop(columns_to_drop, axis=1, inplace= True)

## Removing odds column with too much null values


Some odds column have more null values than other. The odds are very important in the model and just replacing with the median or the mean  could have a negative effect on our predicitions. 

In [7]:
odds_columns = df.iloc[:,23:]
odds_columns.isnull().sum()
odds_columns_to_drop = [c for c in odds_columns.columns if odds_columns[c].isnull().sum() >1]
odds_columns.drop(odds_columns_to_drop, axis=1, inplace=True)

## Creating numerical value for the full time result. 
The full time result (FTR) column is a string but in order to process it later we need to encode it as a numerical value. 

In [8]:
def numerize_ftr_htr(ftr):
    if ftr == "D":
        return 0
    elif ftr == "A":
        return 2
    elif ftr == "H":
        return 1
    else :
        return None
df["Numerical_htr"] = df["HTR"].apply(numerize_ftr_htr)
df["Numerical_ftr"] = df["FTR"].apply(numerize_ftr_htr)

## Creating average odd for home win/draw/away win

In [10]:
home_odds  = [c for c in odds_columns if c[-1] == "H"]
away_odds = [c for c in odds_columns if c[-1]=="A"]
draw_odds = [c for c in odds_columns if c[-1] =="D"]

In [11]:
odds_columns["home_odds_mean"] = odds_columns[home_odds].apply(np.mean, axis=1)

In [12]:
odds_columns["away_odds_mean"] = odds_columns[away_odds].apply(np.mean, axis=1)

In [13]:
odds_columns["draw_odds_mean"] = odds_columns[draw_odds].apply(np.mean, axis=1)

In [17]:
odds_mean = odds_columns[["home_odds_mean", "away_odds_mean","draw_odds_mean"]]

In [20]:
df.drop(['B365H', 'B365D', 'B365A', 'BWH', 'BWD',
       'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA',
       'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'Bb1X2', 'BbMxH', 'BbAvH',
       'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5',
       'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH',
       'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA'], axis=1, inplace=True)

In [22]:
new_df = pd.concat([df, odds_columns], axis=1)

In [24]:
new_df.to_csv('assets/data/clean_data.csv', index=None)

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'Numerical_htr', 'Numerical_ftr', 'B365H',
       'B365D', 'B365A', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH',
       'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA', 'home_odds_mean',
       'away_odds_mean', 'draw_odds_mean'],
      dtype='object')